# Factory System Discrete Event Simulator

https://www.youtube.com/watch?v=G2WftFiBRFg&list=LLGrrsXL3m0dnrQoX3KX0-Dg&index=6&t=0s

In [40]:
import simpy

import numpy as np

In [41]:
np.random.seed(0)

In [42]:
def factory_run(env, repairers, spares):
    
    global cost
    
    cost = 0.0
    
    for i in range(50):
        # Create machine instances
        env.process(operate_machine(env, repairers, spares))
    
    while True:
        cost += 3.75*8*repairers.capacity + 30*spares.capacity
        yield env.timeout(8.0)

In [43]:
def operate_machine(env, repairers, spares):
    global cost
    
    while True:
        yield env.timeout(time_to_failure())
        t_broken = env.now()
        print(f'{t_broken:0.2f} machine broken')
        
        # launch repair process
        yield spares.get(1)
        t_replaced = env.now()
        print(f'{t_replaced:0.2f} machine replaced')
        
        cost += 20*(t_replaced-t_broken)

In [51]:
def repair_machine(env, repairers, spares):
    with repairers.request() as request:
        yield request
        yield env.timeout(generate_repair_time())
        yield spares.put(1)

In [52]:
def time_to_failure():
    return np.random.uniform(132,182)

In [53]:
env = simpy.Environment()

In [54]:
repairers = simpy.Resource(env, capacity=3)

In [55]:
spares = simpy.Container(env, init=20, capacity=20)

In [56]:
env.process(factory_run(env, repairers, spares))

<Process(factory_run) object at 0x29657f80748>

In [57]:
env.run(until=8*5) # unit of time is hours (8 hours over 5 days)